In [4]:
#import matplotlib
#matplotlib.use('Agg')
import matplotlib as mpl
try:
    mpl.use('Qt5Agg')
except ValueError as e:
    print('Error: matplotlib backend\n', e)
    print('Trying:', mpl.get_backend())
    mpl.use(matplotlib.get_backend())

import argparse
import array
import os
import time
from collections import namedtuple
import numpy as np
import random
import matplotlib.pyplot as plt
from matplotlib import cm
import matplotlib.animation as animation
import numba

In [6]:
#first item cell iteself
neumann_n = [(0,0),(0,1),(0,-1),(-1,0),(1,0)]

moore_n = [(0,0),(-1,0),(1,0),(0,1),(0,-1),(1,1),(-1,1),(1,-1),(-1,-1)]

Neighborhood =  { "moore" : moore_n,
                 "neumann": neumann_n
}

#@autojit
#def model(neighbors):
#    if np.sum(neighbors) > 5 or np.sum(neighbors) == 4:
#        return 1
#    else:
#        return 0
    

#Neural Activity
@numba.jit(nopython=True)
def R(x):
    if x == 1:
        return 2
    else:
        return 0
    
@numba.jit(nopython=True)
def D(x, l):
    k = 0
    if 1 in l:
        k = 1
    else:
        k = 0
    if x == 0:
        return k
    else:
        return 0
    
@numba.jit(nopython=True)
def model(neighbors, cell_1):
    cell = neighbors[0]
    return ((R(cell) + D(cell, neighbors[1:]) - cell_1))%3


#cyclic space
#m = 2
#@autojit
#def model(neighbors):
#    cell = neighbors[0]
#    if cell < m:
#        if (cell + 1) in neighbors[1:]:
#            return cell + 1
#        else:
#            return cell
#    elif cell == m:
#        if 0 in neighbors[1:]:
#            return 0
#        else:
#            return cell
#    else:
#        return cell
            

In [17]:
class Grid(object):
    def __init__(self, gridsize, n_steps, rule, neighborhood, n_state = 2, init_state='random'):
        self.Z = np.zeros((gridsize, gridsize),"uint8")
        if init_state == 'random':
            self.Z = np.random.randint(n_state, size=(gridsize, gridsize))
        else:
            self.Z[gridsize // 2, gridsize // 2] = 1
        self.dim = gridsize
        self.steps = n_steps
        self.rule = rule
        self.n_state = n_state
        self.neighborhood = Neighborhood[neighborhood]
        self.t = 0
        self.history = {-1:np.zeros((self.dim, self.dim),"uint8"), 0:self.Z.copy()}
    
    def set_initial2(self):
        self.Z = np.zeros((self.dim, self.dim),"uint8")
        for i in range(50,60):
            for j in range(50,60):
                self.Z[i,j] = random.randint(0,self.n_state-1)
        self.history = {-1:np.zeros((self.dim, self.dim),"uint8"), 0:self.Z.copy()}
        
    def set_initial(self):
        self.Z = np.zeros((self.dim, self.dim),"uint8")
        self.Z[40,40] = 1
        self.Z[49,46] = 1
        self.Z[41,53] = 1
        self.Z[45,45] = 2
        self.history = {-1:np.zeros((self.dim, self.dim),"uint8"), 0:self.Z.copy()}
    
    @numba.jit
    def update(self):
        newZ = self.Z.copy()#np.zeros((self.dim, self.dim),"uint8")
        for i in range(0,self.dim):
            for j in range(0,self.dim):
                neighbors = []
                cell_1 =  self.history[self.t-1][i,j]
                for (dx,dy)in self.neighborhood:
                    x = i + dx
                    y = j + dy
                    if x >= self.dim:
                        x = 0
                    if y >= self.dim:
                        y = 0
                    if x < 0:
                        x = self.dim-1
                    if y < 0:
                        y = self.dim-1    
                    neighbors.append(self.Z[x,y])
                newZ[i,j] = model(neighbors, cell_1)
        
        self.Z = newZ.copy()
        self.t = self.t + 1
        self.history[self.t] = self.Z.copy()
        return self.Z

    def init_plot(self):
        self.fig, self.ax = plt.subplots()
        self.fig.set_size_inches(8, 8, True)

        cmap = cm.RdPu
        cmap.set_over((1., 0., 0.))
        cmap.set_under((0., 0., 1.))
        bounds = list(x for x in range(0, self.n_state))
        norm = mpl.colors.BoundaryNorm(bounds, cmap.N)
        self.im = self.ax.imshow(self.Z, cmap=cmap,animated=True, clim=(0, self.n_state))
        # self.time0 = time.time()
        return self.fig, self.ax
    
    def update_plot(self,i):
        self.im.set_array(self.update())
        # if i == self.steps-1:
            # print(("t= %s seconds " % (time.time() - self.time0)))
        return self.im

In [1]:
n_steps = 2000
G = Grid(140, n_steps, 999, 'neumann', 3, 'random')
G.set_initial2()
fig, ax = G.init_plot()
anim = animation.FuncAnimation(fig,
                                   G.update_plot,
                                   np.arange(1, n_steps),
                                   repeat=False,
                                   blit=False,
                                   interval=100)
plt.axis('off')
plt.tight_layout()

#Writer = animation.writers['ffmpeg']
#writer = Writer(fps=150, bitrate=1800)
#anim.save('moore.mp4', writer=writer)
plt.show()

NameError: name 'Grid' is not defined